In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# https://www.kaggle.com/crowdflower/twitter-airline-sentiment


'wget' is not recognized as an internal or external command,
operable program or batch file.
